In [1]:
from sklearn.preprocessing import StandardScaler

from upbit_control import upbit_control
import tensorflow as tf
from tensorflow import keras


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.models import load_model

# pip install tensorflow==2.13




import pandas as pd
import numpy as np

In [11]:
upbit = upbit_control()

In [9]:

class coin_pred:
    def __init__(self):

        self.upbit = upbit_control()
        self.window = 200
        ######### 코인이름 임시입니다.
    def pre_processing(self, coin_name="KRW-BTC"):
        # 롤링평균 과거 200개 값 필요합니다.
        
        # 지금시간의 코인데이터 반환
        now_coin = upbit.request_upbit_now_400_data(coin_name=coin_name)
        ### 모델이 요구하는 형태로 변환
        now_coin.drop(columns=["market", "candle_date_time_utc","candle_date_time_kst","unit" ], inplace=True)
        now_coin["timestamp"] = now_coin["timestamp"].round(-3)
        # 롤링평균& 데이터 생성 
        rol_mean = now_coin["trade_price"].rolling(self.window).mean()
        rol_half_mean = now_coin["trade_price"].rolling(int(self.window/2)).mean()
        rol_std = now_coin["trade_price"].rolling(self.window).std()
        rol_half_std = now_coin["trade_price"].rolling(int(self.window/2)).std()
        temp_rol = pd.concat([rol_mean, rol_half_mean, rol_std, rol_half_std], axis=1)
        temp_rol.columns = ["rol_mean","rol_half_mean","rol_std","rol_half_std"]
        now_coin = pd.concat([now_coin, temp_rol], axis=1)
        # 윈도우 크기만큼 절단
        now_coin = now_coin[self.window:]
        now_coin.reset_index(drop=True, inplace=True)
        # 데이터 스케일링
        ss = StandardScaler()
        now_coin = ss.fit_transform(now_coin)
        ## LSTM이 요구하는 형태로 변환
        now_coin_np = np.zeros((len(now_coin) - self.window +1, now_coin.shape[0], now_coin.shape[1]), dtype=np.float32)
        for index in range(self.window, len(now_coin) +1):
            now_coin_np[index - self.window] = now_coin[index - self.window : index]
        
        # model_make
        coin_full_name = "KRW-NEO_네오"
        # 모델 가져오기
        model = keras.Sequential()

        model.add(keras.layers.LSTM(64, activation='tanh', input_shape = (now_coin_np.shape[1],now_coin_np.shape[2]) , return_sequences=True ))
        model.add(keras.layers.LSTM(64, activation='tanh'  ,return_sequences = False))
        model.add(keras.layers.Dense(32, activation="relu"))
        model.add(keras.layers.Dense(1, activation="linear"))
        
        RMSprop = keras.optimizers.RMSprop(learning_rate=0.01)
        model.compile(optimizer= RMSprop, loss='mae', metrics=["mse"])

        # model = load_model(f'./model/LSTMEND{coin_name}.keras')
        model.load_weights(f"./model/LSTM_{coin_full_name}.keras")
            
        return  now_coin_np, model
    def predict(self,   coin_name = None):
        pass# 모델 가져오기
        
        
if __name__ == "__main__":
    pred = coin_pred()
    now_coin_np, model = pred.pre_processing()
    now_coin_np, model
    




AttributeError: module 'tensorflow' has no attribute 'keras'

In [39]:
model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 200, 64)           19456     
                                                                 
 lstm_23 (LSTM)              (None, 64)                33024     
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 1)                 33        
                                                                 
Total params: 54593 (213.25 KB)
Trainable params: 54593 (213.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
